In [1]:
import sys
sys.path.insert(1, '../../functions/')

from settings import runcry

from crystal_io import read_input
from crystal_io import write_input

from extract_info import final_energy
from extract_info import band_gap

from settings import clean_wf

import re
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import time

In [2]:
#directory where the input is saved
directory = '../data' 

#name of the input (including the extension)
input_name = 'mgo.d12' 
file_path = join(directory,input_name)

#the functionals are specified as a list. If both correlation and exchange functionals are specified, 
#then this is given as a list of lists.
functionals = [['PBE','PBE'],'B3LYP']

#Specify if this is a spin polarised calculation
spin_pol = False

#Run the calculation? (True = run , False = only write the inputs, 'Analyse' = analyse results of a previous calculation)
run = True

#Delete the wave function files after running the calculation
clean = True

In [3]:
#Read the input and store it in its several blocks
geom_block,optgeom_block,bs_block,func_block,scf_block = read_input(file_path)

In [4]:
#This is the function that runs the calculation (move to the following cell to analuyse the results)
def fc(directory,input_name,geom_block,bs_block,func_block,scf_block,functionals,spin_pol,run,clean):
    files_not_found = []
    energy = []
    for i,functional in enumerate(functionals): 
        if type(functional) == str:
            input_convergence = join(directory,input_name[:-4]+'_%s.d12'%(functional))
        elif type(functional) == list:   
            input_convergence = join(directory,input_name[:-4]+'_%s_%s.d12'%(functional[0],functional[1]))  
        if run != 'Analyse':
            for j,line in enumerate(func_block):
                if re.match(r'DFT',line): 
                    if 'SPIN' in func_block[j+1]: 
                        n = j+2
                        spin_pol = True
                    else:
                        n = j+1
                        spin_pol = False
                    if type(functional) == str:
                        func_block[n] = str(functional)+'\n'
                    elif type(functional) == list:
                        func_block[n] = 'EXCHANGE\n' + functional[0] + '\n' + 'CORRELAT\n' + functional[1] + '\n'            
            write_input(input_convergence,geom_block,bs_block,func_block,scf_block)
        if run == True:
            time_0 = time.time()
            runcry(input_convergence[:-4])
            time.sleep(3)
            E_final = final_energy(input_convergence[:-3]+'out')
            Band_gap = band_gap(input_convergence[:-3]+'out',spin_pol)
            if type(E_final) == float:   
                if type(functional) == str:
                    energy.append([functional,float(E_final),float(Band_gap),time.time()-time_0]) 
                elif type(functional) == list:
                    energy.append([functional[0]+'_'+functional[1],float(E_final),float(Band_gap),time.time()-time_0])
            else:
                if type(functional) == str:
                    files_not_found.append(input_name[:-4]+'_%s.out'%(functional)) 
                elif type(functional) == list:
                    files_not_found.append(input_name[:-4]+'_%s_%s.out'%(functional[0]+'_'+functional[1])) 
            if clean == True:
                clean_wf(directory,input_convergence)
        if run == 'Analyse':
            E_final = final_energy(input_convergence[:-3]+'out')
            Band_gap = band_gap(input_convergence[:-3]+'out',spin_pol)
            if type(E_final) == float:   
                if type(functional) == str:
                    energy.append([functional,float(E_final),float(Band_gap)]) 
                elif type(functional) == list:
                    energy.append([functional[0]+'_'+functional[1],float(E_final),float(Band_gap)])
            else:
                if type(functional) == str:
                    files_not_found.append(input_name[:-4]+'_%s.out'%(functional)) 
                elif type(functional) == list:
                    files_not_found.append(input_name[:-4]+'_%s_%s.out'%(functional[0]+'_'+functional[1])) 
                
    if run == True:
        energy_arr = np.array(energy)
        energy_df = pd.DataFrame(energy, columns =['Functional','Energy','Band Gap (eV)','Time'])
        energy_df.set_index('Functional', inplace=True)
        pd.set_option('display.float_format', '{:.3E}'.format)
    
    if run == 'Analyse':
        energy_arr = np.array(energy)
        energy_df = pd.DataFrame(energy, columns =['Functional','Energy','Band Gap (eV)'])
        energy_df.set_index('Functional', inplace=True)
        pd.set_option('display.float_format', '{:.3E}'.format)
        
        if len(files_not_found) > 0:
            print('The following outputs were not found: '+','.join(files_not_found))

    if run != False:
        return energy_df, energy_arr
    else:
        return 'Inputs saved'
          

### The result is in the cells below

In [5]:
#call the fc function to run the calcaulation/prepare the inputs/analyse the results
#The final energy and band gap is saved as a Pandas dataframe (displayed in the cell below) and as a numpy array.
if run != False:
    energy_df, energy_arr = fc(directory,input_name,geom_block,bs_block,func_block,scf_block,functionals,spin_pol,run,clean)
else:
    display(fc(directory,input_name,geom_block,bs_block,func_block,scf_block,functionals,spin_pol,run,clean))

KeyboardInterrupt: 